# Implimenting A Dense layer from scratch

In this Notebook I have Implimented a Dense layer from scratch which has transformation equation of Z = WT1.X^2 + WT2.X + b , which is inherently a non-Linear transformation.

## Imports

In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np

## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [4]:
# inherit from this base class
from tensorflow.keras.layers import Layer

# Simple Dense layer with the equation Z = WT.X + b
class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

# My dense layer with the equation Z = WT1.X^2 + WT2.X + b
class MyDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(MyDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # Initialize the weights1
        w1_init = tf.random_normal_initializer()
        self.w1 = tf.Variable(name="kernel1",
            initial_value=w1_init(shape=(input_shape[-1], self.units),dtype='float32'),trainable=True)

        # Initialize the weights2
        w2_init = tf.random_normal_initializer()
        self.w2 = tf.Variable(name="kernel2",
            initial_value=w2_init(shape=(input_shape[-1], self.units),dtype='float32'),trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(tf.math.square(inputs), self.w1) + tf.matmul(inputs, self.w2) + self.b

Now we can use our custom layer like below:

In [8]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

# declare an instance of the class
my_dense1 = MyDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense1(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense1.variables)

[<tf.Variable 'simple_dense_4/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.00651846]], dtype=float32)>, <tf.Variable 'simple_dense_4/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]
[<tf.Variable 'my_dense_2/kernel1:0' shape=(1, 1) dtype=float32, numpy=array([[0.06181852]], dtype=float32)>, <tf.Variable 'my_dense_2/kernel2:0' shape=(1, 1) dtype=float32, numpy=array([[-0.03967683]], dtype=float32)>, <tf.Variable 'my_dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Let's then try using it in simple network:

In [10]:
# Simple Dense
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=1000,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

# Quadratic Dense
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)


# use the Sequential API to build a model with our custom layer
my_layer = MyDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=1000,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

[[18.999895]]
[<tf.Variable 'sequential_3/simple_dense_6/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9999847]], dtype=float32)>, <tf.Variable 'sequential_3/simple_dense_6/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9999532], dtype=float32)>]
[[19.001118]]
[<tf.Variable 'sequential_4/my_dense_4/kernel1:0' shape=(1, 1) dtype=float32, numpy=array([[1.9709993e-05]], dtype=float32)>, <tf.Variable 'sequential_4/my_dense_4/kernel2:0' shape=(1, 1) dtype=float32, numpy=array([[1.9999075]], dtype=float32)>, <tf.Variable 'sequential_4/my_dense_4/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9999282], dtype=float32)>]
